In [1]:
%cd ~/code/projects/nn-play/
from structured_container import DataContainer

/home/ira/code/projects/nn-play


In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
BATCH_SIZE = 32
EP_LEN = 100
nc = 1
ndf = 32
ngf = 32
nz = 64


In [4]:
data_test = DataContainer('data-balls/bounce-valid.pt', batch_size=BATCH_SIZE, ep_len_read=EP_LEN)
data_test.populate_images()

Loading data-balls/bounce-valid.pt


In [5]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
class _netD(nn.Module):
    
    def __init__(self):
        super(_netD, self).__init__()
        self.nn_seq = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 3, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        output = self.nn_seq(x)

        return output.view(-1, 1)    
    

In [7]:
netD = _netD()
netD.apply(weights_init)
x = Variable(torch.randn(1,nc,28,28))
out = netD(x)
out

Variable containing:
 0.6079
[torch.FloatTensor of size 1x1]

In [8]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.nn_seq = nn.Sequential(
            # input is Z, going into a convolution
#             nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
#             nn.BatchNorm2d(ngf * 8),
#             nn.ReLU(True),
#             # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 2, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            # nn.Tanh()
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )

    def forward(self, x):
        output = self.nn_seq(x)
        return output


In [9]:
netG = _netG()
netG.apply(weights_init)
x = Variable(torch.randn(1, nz, 1, 1))
out = netG(x)
out;

In [10]:
criterion = nn.BCELoss()

input = torch.FloatTensor(BATCH_SIZE, nc, 28, 28)
noise = torch.FloatTensor(BATCH_SIZE, nz, 1, 1)
fixed_noise = torch.FloatTensor(BATCH_SIZE, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(BATCH_SIZE)
real_label = 1
fake_label = 0

In [11]:
netD.cuda()
netG.cuda()
criterion.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

In [12]:
input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)

In [13]:
optimizerD = optim.Adam(netD.parameters(), lr=0.0002)
optimizerG = optim.Adam(netG.parameters(), lr=0.0002)

In [17]:
def train_cycle():
    x = data_test.get_n_random_images(BATCH_SIZE)
    x = x.transpose((0, 3, 1, 2))
    x = torch.FloatTensor(x)

    ############################
    # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
    ###########################
    # train with real
    netD.zero_grad()
    real_cpu = x
    batch_size = real_cpu.size(0)
    input.data.resize_(real_cpu.size()).copy_(real_cpu)
    label.data.resize_(batch_size).fill_(real_label)

    output = netD(input)
    errD_real = criterion(output, label)
    errD_real.backward()
    D_x = output.data.mean()
    print("D_x", D_x)

    # train with fake
    noise.data.resize_(batch_size, nz, 1, 1)
    noise.data.normal_(0, 1)
    fake = netG(noise)
    label.data.fill_(fake_label)
    output = netD(fake.detach())
    errD_fake = criterion(output, label)
    errD_fake.backward()
    D_G_z1 = output.data.mean()
    errD = errD_real + errD_fake
    optimizerD.step()
    
    print("D_G_z1", D_G_z1)
    
    ############################
    # (2) Update G network: maximize log(D(G(z)))
    ###########################
    netG.zero_grad()
    label.data.fill_(real_label)  # fake labels are real for generator cost
    output = netD(fake)
    errG = criterion(output, label)
    errG.backward()
    D_G_z2 = output.daqta.mean()
    optimizerG.step()


In [64]:
train_D()

D_x 0.9787691831588745
D_G_z1 0.000985470018349588


In [ ]:
def train_G()